# Setup & Data Load

In [1]:
# Import necessary modules and set options
import pandas as pd
import numpy as np
from random import shuffle

# Load data
data = pd.read_csv("prostate_data", sep = "\t")
print(data.head())

# Train-test split
y_train = np.array(data[data.train == "T"]['lpsa'])
y_test = np.array(data[data.train == "F"]['lpsa'])
X_train = np.array(data[data.train == "T"].drop(['lpsa', 'train'], axis=1))
X_test = np.array(data[data.train == "F"].drop(['lpsa', 'train'], axis=1))

# Set cross-validation indices
num_obs_9_folds = np.floor(X_train.shape[0] / 10)
num_obs_last_fold = X_train.shape[0] - (10 * num_obs_9_folds)
idxs = ([x for x in range(9)] * int(num_obs_9_folds)) + ([9] * int(num_obs_last_fold))
shuffle(idxs)

     lcavol   lweight  age      lbph  svi       lcp  gleason  pgg45      lpsa  \
0 -0.579818  2.769459   50 -1.386294    0 -1.386294        6      0 -0.430783   
1 -0.994252  3.319626   58 -1.386294    0 -1.386294        6      0 -0.162519   
2 -0.510826  2.691243   74 -1.386294    0 -1.386294        7     20 -0.162519   
3 -1.203973  3.282789   58 -1.386294    0 -1.386294        6      0 -0.162519   
4  0.751416  3.432373   62 -1.386294    0 -1.386294        6      0  0.371564   

  train  
0     T  
1     T  
2     T  
3     T  
4     T  


In [30]:
# Center data

# The features must first be scaled to have mean zero and  variance 96 (=n)
#before the analyses in Tables 3.1 and beyond.  That is, if x is the  96 by 8 matrix
#of features, we compute xp <- scale(x,TRUE,TRUE)


X_train_means = X_train.mean(axis=0)
X_train_centered = X_train - X_train_means[None, :]

# Linear Regression

In [14]:
# Cross-validate to find MAE standard errors
mae_cv = np.array([])
for fold in range(9):
    
    # Split training data into 9 training folds and 1 testing fold
    train_indices = [i for i, e in enumerate(idxs) if e != fold]
    test_indices = [i for i, e in enumerate(idxs) if e == fold]
    current_X_train = X_train[train_indices, :]
    current_X_test = X_train[test_indices, :]
    current_y_train = y_train[train_indices]
    current_y_test = y_train[test_indices]

    # Calculate the coefficients based on the training 9 folds
    betas = np.matmul(np.linalg.inv(np.matmul(current_X_train.T,  current_X_train)),
                      np.matmul(current_X_train.T, current_y_train))
    
    # Predict test data (holdout fold)
    y_pred = np.matmul(current_X_test, betas)
    
    # Get Mean Absolute Error for the current fold
    mae_cv = np.append(mae_cv, np.mean(np.abs(y_pred - current_y_test)))

# Predict test data and calculate out-of-sample MAE
betas = np.matmul(np.linalg.inv(np.matmul(X_train.T,  X_train)), np.matmul(X_train.T, y_train))
y_pred = np.matmul(X_test, betas)
mae = np.mean(np.abs(y_pred - y_test))
    
# Get cross-validation based estimates of MAE standard error
# (standarddeviation of the mean cross-validated error)
mae_se = np.std(mae_cv)

print('Linear Regression MAE: {}'.format(np.round(mae, 2)))
print('Linear Regression MAE SE: {}'.format(np.round(mae_se, 2)))

Linear Regression MAE: 0.52
Linear Regression MAE SE: 0.21
